<a href="https://colab.research.google.com/github/danadria/Skills-Lab-Introduction-to-Transformers-BERT-and-Explainable-NLP/blob/main/skills_lab_sentiment_analysis_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical 10: Transformers for Sentiment Analysis
#### Daniel Anadria

<img src="img/uu_logo.png" alt="logo" align="right" title="UU" width="50" height="20" />


#### Applied Text Mining - Utrecht Summer School

**Practical is under construction, parts remain to be annotated.**


In this practical, we are going to perform sentiment analysis of movie reviews using a transformer model, have a look under its hood, and try to explain the model predictions using SHAP.

Our model of choice is [DistilBERT](https://medium.com/huggingface/distilbert-8cf3380435b5), a light-weight transformer whose performance is comparable to Google's [BERT base model](https://arxiv.org/abs/1810.04805).



## Overview

In Part 1, we will use an off-the-shelf sentiment analysis pipeline from the Hugging Face [`transformers`](https://huggingface.co/docs/transformers/index) module to classify two movie reviews.

In Part 2, we will dissasemble the sentiment analysis pipeline by performing the same analysis as in Part 1 step-by-step.

In Part 3, we will open the black box and explore which tokens were most important for DistilBERT's sentiment classification. We do this using [Shapley Additive Explanations \(SHAP\)](https://arxiv.org/abs/1705.07874).

In Part 4, we will fine-tune DistilBERT on the IMDB movie review dataset.

## Prepare the Colab Environment

**Fine-tuning a transformer model is quite resource-intensive! Switch your runtime type to GPU T4 under Runtime > Change runtime type.**

**Running this practical requires a more recent version of the `accelerate` package than installed by default in Google Colab. Run the code below to upgrade `accelerate`.**

In [1]:
!pip install -q -U accelerate # update accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.6 MB/s eta 0:00:00


**Now restart your runtime under Runtime > Restart runtime or by pressing `ctrl + M .` (a pop-up prompt should appear)**

**All set?**

## Part 1: Off-the-shelf sentiment analysis pipeline

Since sentiment analysis is a popular application, there are off-the-shelf pipelines which we can use to quickly classify documents by sentiment. One such pipeline is part of the Hugging Face `transformers` module.



<blockquote> 🤗 Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch.

[ ... ]

The pipelines are a great and easy way to use models for inference. These pipelines are objects that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering. See the task summary for examples of use.

[ ... ]

The `pipeline()` is the most powerful object encapsulating all other pipelines.


</blockquote>



In [1]:
!pip install -q transformers
!pip install -q Xformers
from transformers import pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.3 MB/s eta 0:00:00


In [3]:
from transformers import set_seed
set_seed(137)

Our model will be distilbert base uncased. Uncased means that the model disregards casing (upper or lower case). Our distilbert version has been fine-tuned for binary sentiment classification using the [Stanford Sentiment Treebank](https://arxiv.org/abs/cs/0506075) (SST-2; Pang and Lee, 2005) corpus. Hence, we can use it off-the-shelf.

Pre-trained BERT models are available for many different natural language processing tasks based on the General Language Understanding Evaluation (GLUE) [benchmark resources](https://huggingface.co/datasets/glue).


To showcase how to use the sentiment analysis pipeline, we will compare two relatively complex IMDB reviews of Mark Mylod's 2022 movie *The Menu* (2022). Load the following two reviews:

In [4]:
review1 = "The Menu isn't the first to satirise the rich and their incompetence and isn't saying anything new but that definitely doesn't prevent it from being a great satire that pokes fun at everything it can in ways that are often consistently funny, playful and extremely stylish. Ralph Fiennes gives a terrific performance full of awkward unease that only enhances his commanding screen presence. Anya Taylor-Joy is a perfect audience surrogate amongst a sea of deliberately unlikeable characters of which the best is Nicholas Hoult whose almost too good at making his character hilariously pathetic. Mark Mylod's direction is excellent, the film has more than enough visual style to match the pretentiousness of its characters and is really good at building tension. The music by Colin Stetson is fantastic, striking a unusual balance between beautiful and unnerving."


In [5]:
review2 = "This looked like an interesting film based on the trailer and the first half of it was just that. The tension and suspense was building nicely. There were little dribs and drabs and hints of what might be coming without being too obvious. The acting from everyone in the film was good. Even supporting characters with only a few lines. Were well realized I remember thinking that I couldn't wait to see where it was all going. Sadly it didn't really go anywhere. It all unwound in the second half. The acting was still on but the writing failed. That's the most i can say without giving up any spoilers. And that was extra disappointing because the first half was so good. This Menu did not deliver the meal as advertised."


You can skim the reviews.

In [6]:
print(review1)

The Menu isn't the first to satirise the rich and their incompetence and isn't saying anything new but that definitely doesn't prevent it from being a great satire that pokes fun at everything it can in ways that are often consistently funny, playful and extremely stylish. Ralph Fiennes gives a terrific performance full of awkward unease that only enhances his commanding screen presence. Anya Taylor-Joy is a perfect audience surrogate amongst a sea of deliberately unlikeable characters of which the best is Nicholas Hoult whose almost too good at making his character hilariously pathetic. Mark Mylod's direction is excellent, the film has more than enough visual style to match the pretentiousness of its characters and is really good at building tension. The music by Colin Stetson is fantastic, striking a unusual balance between beautiful and unnerving.


In [7]:
print(review2)

This looked like an interesting film based on the trailer and the first half of it was just that. The tension and suspense was building nicely. There were little dribs and drabs and hints of what might be coming without being too obvious. The acting from everyone in the film was good. Even supporting characters with only a few lines. Were well realized I remember thinking that I couldn't wait to see where it was all going. Sadly it didn't really go anywhere. It all unwound in the second half. The acting was still on but the writing failed. That's the most i can say without giving up any spoilers. And that was extra disappointing because the first half was so good. This Menu did not deliver the meal as advertised.


 What is your guess of the sentiment of the following reviews? On the scale 1-10, what rating do you think the respective authors gave the movie?

**1. Set up and fit a sentiment analysis pipeline to predict the sentiment of the two reviews. Define the model as `'distilbert-base-uncased-finetuned-sst-2-english'`**

In [8]:
sentiment_pipeline = pipeline("sentiment-analysis", model = 'distilbert-base-uncased-finetuned-sst-2-english')

In [9]:
sentiment_pipeline(review1) # predict sentiment

[{'label': 'POSITIVE', 'score': 0.9993577599525452}]

In [10]:
sentiment_pipeline(review2) # predict sentiment

[{'label': 'NEGATIVE', 'score': 0.9622442722320557}]

For each review, we see the output label and the associated probability.

Do you agree with model predictions?

Here is the ground truth:

[Review 1](https://www.imdb.com/review/rw8682076/?ref_=tt_urv) is a positive review with a rating of 8/10.

[Review 2](https://www.imdb.com/review/rw8693249/?ref_=tt_urv) is a negative review with a rating of 4/10.

For each review, we see the output label, and the associated probability.

Now we are going to show you how to build your own sentiment analysis pipeline from scratch. In practice you can use the already existing one, but it is helpful to understand the steps associated with setting up a transformer-based pipeline.



## Part 2: Sentiment Analysis Pipeline - Deconstructed

Same analysis on the same two reviews - this time step-by-step.

**2. Define the tokenizer and model. For the tokenizer, use the pretrained DistilBERT tokenizer (`DistilBertTokenizer.from_pretrained`) and for the model use `"distilbert-base-uncased-finetuned-sst-2-english"`.**

In [11]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

**3. Tokenize the `review1` and `review2` objects. Pad and truncate the sequences, and return PyTorch (`pt`) tensors. Save the output object as `encoding`.**


In [12]:
encoding = tokenizer([review1, review2], padding = True, truncation = True, return_tensors = 'pt') # tokenize the reviews

BERT and several other transformer models use tokenizers based on [WordPiece](https://huggingface.co/learn/nlp-course/chapter6/6?fw=pt), a subword tokenization algorithm. The main advantage of a subword tokenizer is that it interpolates between word-based and character-based tokenization. Common words get a slot in the vocabulary, but the tokenizer can fall back to word pieces and individual characters for unknown words.

Since batched inputs (our reviews) are of different lengths, they cannot be converted to fixed-size tensors to befed to the model.

There are two main strategies for solving this problem -- *padding* and *truncation*.

In order to create rectangular tensors from batches of varying lengths, padding adds a special padding token to ensure shorter sequences will have the same length as either the longest sequence in a batch or the maximum length accepted by the model. Truncation works in the other direction by truncating long sequences.

`padding = True`: pad to the longest sequence in the batch (no padding is applied if you only provide a single sequence).

`truncation = True`: truncate to a maximum length specified by the max_length argument or the maximum length accepted by the model if no max_length is provided (max_length=None).

**4. Inspect the `encoding` object by prining the first reveiw's input ids.**

In [13]:
print(encoding['input_ids'][0]) # first review's input_ids

tensor([  101,  1996, 12183,  3475,  1005,  1056,  1996,  2034,  2000,  2938,
        15735,  3366,  1996,  4138,  1998,  2037,  4297, 25377, 12870,  5897,
         1998,  3475,  1005,  1056,  3038,  2505,  2047,  2021,  2008,  5791,
         2987,  1005,  1056,  4652,  2009,  2013,  2108,  1037,  2307, 18312,
         2008, 26202,  2015,  4569,  2012,  2673,  2009,  2064,  1999,  3971,
         2008,  2024,  2411, 10862,  6057,  1010, 18378,  1998,  5186,  2358,
         8516,  4509,  1012,  6798, 10882, 24336,  2015,  3957,  1037, 27547,
         2836,  2440,  1997,  9596, 27880,  2008,  2069, 11598,  2015,  2010,
         7991,  3898,  3739,  1012, 21728,  4202,  1011,  6569,  2003,  1037,
         3819,  4378,  7505, 21799,  5921,  1037,  2712,  1997,  9969,  4406,
         3085,  3494,  1997,  2029,  1996,  2190,  2003,  6141,  7570, 11314,
         3005,  2471,  2205,  2204,  2012,  2437,  2010,  2839, 26316,  2135,
        17203,  1012,  2928,  2026,  4135,  2094,  1005,  1055, 

We see that BERT assigns a unique id to each token (`input_ids`).

**5. Convert first review's input ids to tokens using `convert_ids_to_tokens` to see how the text got tokenized.**

In [14]:
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0])) # first review's tokens

['[CLS]', 'the', 'menu', 'isn', "'", 't', 'the', 'first', 'to', 'sat', '##iri', '##se', 'the', 'rich', 'and', 'their', 'inc', '##omp', '##ete', '##nce', 'and', 'isn', "'", 't', 'saying', 'anything', 'new', 'but', 'that', 'definitely', 'doesn', "'", 't', 'prevent', 'it', 'from', 'being', 'a', 'great', 'satire', 'that', 'poke', '##s', 'fun', 'at', 'everything', 'it', 'can', 'in', 'ways', 'that', 'are', 'often', 'consistently', 'funny', ',', 'playful', 'and', 'extremely', 'st', '##yl', '##ish', '.', 'ralph', 'fi', '##enne', '##s', 'gives', 'a', 'terrific', 'performance', 'full', 'of', 'awkward', 'unease', 'that', 'only', 'enhance', '##s', 'his', 'commanding', 'screen', 'presence', '.', 'anya', 'taylor', '-', 'joy', 'is', 'a', 'perfect', 'audience', 'sur', '##rogate', 'amongst', 'a', 'sea', 'of', 'deliberately', 'unlike', '##able', 'characters', 'of', 'which', 'the', 'best', 'is', 'nicholas', 'ho', '##ult', 'whose', 'almost', 'too', 'good', 'at', 'making', 'his', 'character', 'hilarious', 

Note that BERT-based models also operate with special tokens:


| Token      | Token ID | Meaning                                 |
|:----------:|:--------:|:---------------------------------------:|
| `[CLS]`    | `101`    | Beginning of input                     |
| `[SEP]`    | `102`    | End of input or sentence               |
| `[MASK]`   | `103`    | Masked tokens the model should predict |
| `[PAD]`    | `0`      | Padding                                 |
| `[UNK]`    | `100`    | Unknown token not in training data     |

**6. Predict sentiment of the two reviews. In order to do this, import `torch`, and define the `output` object using the model, input ids and attention mask.**

Now we are ready to do some sentiment prediction. We import `torch`, define our model by feeding it `input_ids`, `attention_mask`. The attention mask is a binary tensor indicating the position of the padded indices so that the model does not attend to them.

In [16]:
# prediction of sentiment
import torch
output = model(input_ids = encoding['input_ids'], attention_mask = encoding['attention_mask'])

In [17]:
print("Predicted logits:\n\n", output['logits']) # logits

Predicted logits:

 tensor([[-3.5648,  3.7852],
        [ 1.8161, -1.4221]], grad_fn=<AddmmBackward0>)


In [18]:
print("Predicted probabilities:\n\n", torch.nn.functional.softmax(output['logits'], dim=-1)) # from logits to probabilities


Predicted probabilities:

 tensor([[6.4221e-04, 9.9936e-01],
        [9.6224e-01, 3.7756e-02]], grad_fn=<SoftmaxBackward0>)


In [19]:
prediction = torch.argmax(output['logits'], 1) # from logits to binary class
print("Predicted classes:\n", prediction)

Predicted classes:
 tensor([1, 0])


We see that our output sentiment and probabilities are the same as when using off-the-shelf sentiment classification pipeline.

## Part 3: Feature importance with SHAP

Now that we have classified our two reviews, we might want to explain DistilBERT's predictions using [Shapley Additive Values (SHAP)](https://shap.readthedocs.io/en/latest/).


**7. Install the `shap` module, import `shap.Explainer` and feed it the `sentiment_pipeline` model. Pass the two movie reviews as input for the explainer.**

*Note. The computation of Shapley values for DistilBERT explaining our two reviews should take about 4 minutes, but can be very computationally intensive in most real life applications.*

In [20]:
!pip install -q shap
import shap

explainer = shap.Explainer(sentiment_pipeline)

shap_values = explainer([review1, review2])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 11.3 MB/s eta 0:00:00


  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer:  50%|█████     | 1/2 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

Partition explainer: 3it [05:15, 157.53s/it]


**8. A nice thing about the `shap` module is that it comes with a built-in visualizer. Use `shap.plots.text` to visualize the shap values for the first and the second movie review.**

In [21]:
shap.plots.text(shap_values[0]) # first review

In [22]:
shap.plots.text(shap_values[1]) # second review

## Part 4: Fine-tuning BERT using the IMDb dataset

Now let's do a sentiment analysis of the IMDB dataset using the sentiment analysis pipeline.

Since the DistilBERT we are using was trained on the Stanford Sentiment Treebank (SST) dataset, we also fine-tune our model for IMDb movie reviews. In practice, this might not be neccesary for this particular application, but might be good to see how it can be done.


In [23]:
!pip install -q datasets
!pip install -q transformers
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.7 MB/s eta 0:00:00


In [24]:
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import evaluate
import numpy as np

**9. Load the IMDb dataset and sample 10% of the `train`, `test`, and `unsupervised`.**

In [25]:
imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
imdb["test"][0] # examine the first instance in test

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [27]:
imdb.shape # inspect dimensions full data

{'train': (25000, 2), 'test': (25000, 2), 'unsupervised': (50000, 2)}

Because fine-tuning on the entire IMDb dataset would be too resource-intensive to run in this practical, we will work with a randomly sampled 10% of the original train and test dataset size.

In [28]:
imdb_sample = imdb
imdb_sample['train'] = imdb['train'].shuffle(seed=42).select(range(int(0.1*len(imdb['train']))))
imdb_sample['test'] = imdb['test'].shuffle(seed=42).select(range(int(0.1*len(imdb['test']))))
imdb_sample['unsupervised'] = imdb['unsupervised'].shuffle(seed=42).select(range(int(0.1*len(imdb['unsupervised']))))

In [29]:
imdb_sample.shape

{'train': (2500, 2), 'test': (2500, 2), 'unsupervised': (5000, 2)}

The next step is to load a DistilBERT tokenizer to preprocess the text field:

In [30]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

Create a preprocessing function to tokenize text and truncate sequences to be no longer than DistilBERT’s maximum input length:

In [31]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

To apply the preprocessing function over the entire dataset, use Datasets map function. You can speed up map by setting `batched=True` to process multiple elements of the dataset at once:

In [32]:
tokenized_imdb = imdb_sample.map(preprocess_function, batched=True)

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Including a metric during training is often helpful for evaluating your model’s performance. You can quickly load a evaluation method with the `evaluate` library. For this task, load the accuracy metric.

In [33]:
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):

    predictions, labels = eval_pred

    predictions = np.argmax(predictions, axis=1)

    return accuracy.compute(predictions=predictions, references=labels)

In [34]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [35]:
model = AutoModelForSequenceClassification.from_pretrained(

    "distilbert-base-uncased-finetuned-sst-2-english", num_labels=2, id2label=id2label, label2id=label2id

)

In [36]:
training_args = TrainingArguments(

    output_dir="tuned_model",

    learning_rate=2e-5,

    per_device_train_batch_size=16,

    per_device_eval_batch_size=16,

    num_train_epochs=2,

    weight_decay=0.01,

    evaluation_strategy="epoch",

    logging_steps = 100,

    save_strategy="epoch",

    load_best_model_at_end=True,

    push_to_hub=False,

)

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_imdb["train"],

    eval_dataset=tokenized_imdb["test"],

    tokenizer=tokenizer,

    compute_metrics=compute_metrics,

)

trainer.train()
trainer.save_model()

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.323600,0.233133,0.912000
2,0.145300,0.294243,0.908400


In [37]:
classifier = pipeline("sentiment-analysis", model="tuned_model")
classifier("The movie was good.")

[{'label': 'POSITIVE', 'score': 0.9992430210113525}]

## Remember: Be on the lookout for bias and other limitations!

Pre-trained transformer models have been made available for many different tasks and by many different people. It is important to be aware that there may be bias and other limitations in the models that could affect your results.

DistilBERT is known to produce biased predictions that target underrepresented populations. For instance, for sentences like This film was filmed in COUNTRY, DistilBERT for binary classification will give radically different probabilities for the positive label depending on the country (0.89 if the country is France, but 0.08 if the country is Afghanistan) when nothing in the input indicates such a strong semantic shift.

See:

[Risks, Limitations and Biases](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english#risks-limitations-and-biases)

[Aurélien Géron's Sentiment Bias Map](https://colab.research.google.com/gist/ageron/fb2f64fb145b4bc7c49efc97e5f114d3/biasmap.ipynb)

In [38]:
sentiment_pipeline("French movie")

[{'label': 'POSITIVE', 'score': 0.9987333416938782}]

In [39]:
sentiment_pipeline("Yemeni movie")

[{'label': 'POSITIVE', 'score': 0.5799139142036438}]

When in doubt fine-tune and use feature importance measures!

## Further reading / materials



*   How to fine-tune a model: https://huggingface.co/docs/transformers/training



## Credits

Many code and quote blocks are adapted from the [HuggingFace Documentation website](https://huggingface.co/docs/). The website contains a lot of additional information and is a great resource for learners.